In [1]:
import numpy as np
from keras.initializers import TruncatedNormal
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import h5py
from keras.callbacks import ModelCheckpoint
import math

s = np.arange(1, 8)
for snr in s:
    # Load Matlab DataSets
    mat = h5py.File('C:\\Users\\yasht\\Desktop\\SAUH Code\\dataset\\Dataset_{}.mat'.format(snr), 'r')
    X = np.array(mat['Preamble_Error_Correction_Dataset']['X'])
    Y = np.array(mat['Preamble_Error_Correction_Dataset']['Y'])
    print('Loaded Dataset Inputs: ', X.shape)
    print('Loaded Dataset Outputs: ', Y.shape)



Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)
Loaded Dataset Inputs:  (2000, 104)
Loaded Dataset Outputs:  (2000, 104)


In [2]:
    # Normalizing Datasets
    scalerx = StandardScaler()
    scalerx.fit(X)
    scalery = StandardScaler()
    scalery.fit(Y)
    XS = scalerx.transform(X)
    YS = scalery.transform(Y)
    # save the mean and standard deviation values to file
    a = np.zeros(104)
    b = np.zeros(104)
    for i in range(scalerx.mean_.shape[0]):
        a[i] = scalerx.mean_[i]
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\mean.dat'.format(snr), a, fmt='%f', newline=',')
    for i in range(scalerx.mean_.shape[0]):
        b[i] = math.sqrt(scalerx.var_[i])
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\std.dat'.format(snr), b, fmt='%f', newline=',')
    for i in range(scalery.mean_.shape[0]):
        a[i] = scalery.mean_[i]
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\mean_o.dat'.format(snr), a, fmt='%f', newline=',')
    for i in range(scalery.mean_.shape[0]):
        b[i] = math.sqrt(scalery.var_[i])
    np.savetxt('C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\DNN{}\\std_o.dat'.format(snr), b, fmt='%f', newline=',')

   

In [4]:
# Split Data into train and test sets
seed = 7
train_X, test_X, train_Y, test_Y = train_test_split(XS, YS, test_size=0.2, random_state=seed)
print('Training samples: ', train_X.shape[0])
print('Testing samples: ', test_X.shape[0])



Training samples:  1600
Testing samples:  400


In [5]:
    # Build the model.
    init = TruncatedNormal(mean=0.0, stddev=0.05, seed=None)
    model = Sequential([
        Dense(units=52, activation='relu', input_dim=104,
              kernel_initializer=init,
              bias_initializer=init),
        Dense(units=104, kernel_initializer=init,
              bias_initializer=init)
    ])

    # Compile the model.
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
    print(model.summary())


    model_path = 'C:\\Users\\yasht\\Desktop\\SAUH Code\\Saved\\LS_DNN_{}.h5'.format(snr)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                5460      
                                                                 
 dense_1 (Dense)             (None, 104)               5512      
                                                                 
Total params: 10,972
Trainable params: 10,972
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
    # This check point saves the best DNN model with highest validation accuracy
    checkpoint = ModelCheckpoint(model_path, monitor='val_acc',
                                 verbose=1, save_best_only=True,
                                 mode='max')
    callbacks_list = [checkpoint]
    # Train the model.
    epoch = 500
    batch_size = 32

    model.fit(train_X, train_Y, epochs=epoch, batch_size=batch_size, verbose=2, validation_split=0.25,  callbacks=callbacks_list)


Epoch 1/500

Epoch 00001: val_acc improved from -inf to 0.10250, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 1s - loss: 0.5233 - acc: 0.0467 - val_loss: 0.1483 - val_acc: 0.1025 - 929ms/epoch - 24ms/step
Epoch 2/500

Epoch 00002: val_acc improved from 0.10250 to 0.16750, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 0.1118 - acc: 0.1142 - val_loss: 0.0832 - val_acc: 0.1675 - 164ms/epoch - 4ms/step
Epoch 3/500

Epoch 00003: val_acc improved from 0.16750 to 0.25500, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 0.0573 - acc: 0.2250 - val_loss: 0.0325 - val_acc: 0.2550 - 143ms/epoch - 4ms/step
Epoch 4/500

Epoch 00004: val_acc improved from 0.25500 to 0.29750, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 0.0222 - acc: 0.2883 - val_loss: 0.0172 - val_acc: 0.2975 - 139ms/epoch - 4ms/step
Epoch 5/500

Epoch 00005: val_acc improved from 0.2975

Epoch 38/500

Epoch 00038: val_acc did not improve from 0.73250
38/38 - 0s - loss: 0.0013 - acc: 0.7483 - val_loss: 0.0014 - val_acc: 0.7175 - 134ms/epoch - 4ms/step
Epoch 39/500

Epoch 00039: val_acc improved from 0.73250 to 0.74500, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 0.0013 - acc: 0.7575 - val_loss: 0.0013 - val_acc: 0.7450 - 170ms/epoch - 4ms/step
Epoch 40/500

Epoch 00040: val_acc did not improve from 0.74500
38/38 - 0s - loss: 0.0012 - acc: 0.7517 - val_loss: 0.0013 - val_acc: 0.7325 - 127ms/epoch - 3ms/step
Epoch 41/500

Epoch 00041: val_acc did not improve from 0.74500
38/38 - 0s - loss: 0.0012 - acc: 0.7650 - val_loss: 0.0012 - val_acc: 0.7450 - 124ms/epoch - 3ms/step
Epoch 42/500

Epoch 00042: val_acc improved from 0.74500 to 0.76000, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 0.0011 - acc: 0.7708 - val_loss: 0.0012 - val_acc: 0.7600 - 131ms/epoch - 3ms/step
Epoch 43/500

Epoch 00043:

38/38 - 0s - loss: 4.8292e-04 - acc: 0.8617 - val_loss: 5.0171e-04 - val_acc: 0.8475 - 150ms/epoch - 4ms/step
Epoch 80/500

Epoch 00080: val_acc did not improve from 0.84750
38/38 - 0s - loss: 4.5465e-04 - acc: 0.8608 - val_loss: 5.5823e-04 - val_acc: 0.8275 - 125ms/epoch - 3ms/step
Epoch 81/500

Epoch 00081: val_acc did not improve from 0.84750
38/38 - 0s - loss: 5.0156e-04 - acc: 0.8458 - val_loss: 4.9315e-04 - val_acc: 0.8425 - 103ms/epoch - 3ms/step
Epoch 82/500

Epoch 00082: val_acc improved from 0.84750 to 0.85000, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 4.4796e-04 - acc: 0.8650 - val_loss: 4.6264e-04 - val_acc: 0.8500 - 155ms/epoch - 4ms/step
Epoch 83/500

Epoch 00083: val_acc improved from 0.85000 to 0.85250, saving model to C:\Users\yasht\Desktop\SAUH Code\Saved\LS_DNN_7.h5
38/38 - 0s - loss: 4.4217e-04 - acc: 0.8625 - val_loss: 4.8170e-04 - val_acc: 0.8525 - 172ms/epoch - 5ms/step
Epoch 84/500

Epoch 00084: val_acc improved from 0

38/38 - 0s - loss: 1.3189e-04 - acc: 0.9183 - val_loss: 1.6486e-04 - val_acc: 0.9375 - 86ms/epoch - 2ms/step
Epoch 122/500

Epoch 00122: val_acc did not improve from 0.93750
38/38 - 0s - loss: 1.4417e-04 - acc: 0.9242 - val_loss: 1.6459e-04 - val_acc: 0.9200 - 69ms/epoch - 2ms/step
Epoch 123/500

Epoch 00123: val_acc did not improve from 0.93750
38/38 - 0s - loss: 1.3262e-04 - acc: 0.9283 - val_loss: 1.2990e-04 - val_acc: 0.9250 - 61ms/epoch - 2ms/step
Epoch 124/500

Epoch 00124: val_acc did not improve from 0.93750
38/38 - 0s - loss: 1.2812e-04 - acc: 0.9242 - val_loss: 1.7139e-04 - val_acc: 0.9250 - 61ms/epoch - 2ms/step
Epoch 125/500

Epoch 00125: val_acc did not improve from 0.93750
38/38 - 0s - loss: 1.2447e-04 - acc: 0.9317 - val_loss: 1.2916e-04 - val_acc: 0.9250 - 61ms/epoch - 2ms/step
Epoch 126/500

Epoch 00126: val_acc did not improve from 0.93750
38/38 - 0s - loss: 1.1335e-04 - acc: 0.9275 - val_loss: 1.3971e-04 - val_acc: 0.9200 - 57ms/epoch - 1ms/step
Epoch 127/500

Epoch 

Epoch 168/500

Epoch 00168: val_acc did not improve from 0.96000
38/38 - 0s - loss: 1.3337e-04 - acc: 0.9400 - val_loss: 1.4819e-04 - val_acc: 0.9325 - 60ms/epoch - 2ms/step
Epoch 169/500

Epoch 00169: val_acc did not improve from 0.96000
38/38 - 0s - loss: 1.0539e-04 - acc: 0.9400 - val_loss: 9.9237e-05 - val_acc: 0.9325 - 55ms/epoch - 1ms/step
Epoch 170/500

Epoch 00170: val_acc did not improve from 0.96000
38/38 - 0s - loss: 9.7131e-05 - acc: 0.9442 - val_loss: 8.3370e-05 - val_acc: 0.9350 - 60ms/epoch - 2ms/step
Epoch 171/500

Epoch 00171: val_acc did not improve from 0.96000
38/38 - 0s - loss: 7.4701e-05 - acc: 0.9500 - val_loss: 7.1212e-05 - val_acc: 0.9400 - 69ms/epoch - 2ms/step
Epoch 172/500

Epoch 00172: val_acc did not improve from 0.96000
38/38 - 0s - loss: 8.3697e-05 - acc: 0.9458 - val_loss: 1.1976e-04 - val_acc: 0.9350 - 62ms/epoch - 2ms/step
Epoch 173/500

Epoch 00173: val_acc did not improve from 0.96000
38/38 - 0s - loss: 1.0064e-04 - acc: 0.9417 - val_loss: 8.4581e-0

...

Epoch 309/500

Epoch 00309: val_acc did not improve from 0.96750
38/38 - 0s - loss: 8.3266e-05 - acc: 0.9525 - val_loss: 1.3590e-04 - val_acc: 0.9400 - 61ms/epoch - 2ms/step
Epoch 310/500

Epoch 00310: val_acc did not improve from 0.96750
38/38 - 0s - loss: 1.7141e-04 - acc: 0.9458 - val_loss: 1.0876e-04 - val_acc: 0.9575 - 58ms/epoch - 2ms/step
Epoch 311/500

Epoch 00311: val_acc did not improve from 0.96750
38/38 - 0s - loss: 1.0318e-04 - acc: 0.9533 - val_loss: 9.6301e-05 - val_acc: 0.9450 - 59ms/epoch - 2ms/step
Epoch 312/500

Epoch 00312: val_acc did not improve from 0.96750
38/38 - 0s - loss: 8.3120e-05 - acc: 0.9575 - val_loss: 8.2617e-05 - val_acc: 0.9500 - 60ms/epoch - 2ms/step
Epoch 313/500

Epoch 00313: val_acc did not improve from 0.96750
38/38 - 0s - loss: 1.3143e-04 - acc: 0.9517 - val_loss: 2.9066e-04 - val_acc: 0.9350 - 59ms/epoch - 2ms/step
Epoch 314/500

Epoch 00314: val_acc did not improve from 0.96750
38/38 - 0s - loss: 2.2545e-04 - acc: 0.9367 - val_loss: 1.8963e-0

Epoch 357/500

Epoch 00357: val_acc did not improve from 0.96750
38/38 - 0s - loss: 3.4372e-05 - acc: 0.9633 - val_loss: 3.8935e-05 - val_acc: 0.9525 - 99ms/epoch - 3ms/step
Epoch 358/500

Epoch 00358: val_acc did not improve from 0.96750
38/38 - 0s - loss: 3.4528e-05 - acc: 0.9617 - val_loss: 4.8342e-05 - val_acc: 0.9600 - 132ms/epoch - 3ms/step
Epoch 359/500

Epoch 00359: val_acc did not improve from 0.96750
38/38 - 0s - loss: 4.8959e-05 - acc: 0.9558 - val_loss: 6.6134e-05 - val_acc: 0.9475 - 129ms/epoch - 3ms/step
Epoch 360/500

Epoch 00360: val_acc did not improve from 0.96750
38/38 - 0s - loss: 4.0913e-05 - acc: 0.9625 - val_loss: 4.9592e-05 - val_acc: 0.9400 - 127ms/epoch - 3ms/step
Epoch 361/500

Epoch 00361: val_acc did not improve from 0.96750
38/38 - 0s - loss: 4.0929e-05 - acc: 0.9558 - val_loss: 4.8484e-05 - val_acc: 0.9525 - 124ms/epoch - 3ms/step
Epoch 362/500

Epoch 00362: val_acc did not improve from 0.96750
38/38 - 0s - loss: 3.8694e-05 - acc: 0.9650 - val_loss: 6.839

Epoch 404/500

Epoch 00404: val_acc did not improve from 0.97000
38/38 - 0s - loss: 1.9827e-04 - acc: 0.9442 - val_loss: 1.2026e-04 - val_acc: 0.9525 - 57ms/epoch - 2ms/step
Epoch 405/500

Epoch 00405: val_acc did not improve from 0.97000
38/38 - 0s - loss: 1.5661e-04 - acc: 0.9425 - val_loss: 1.4091e-04 - val_acc: 0.9450 - 59ms/epoch - 2ms/step
Epoch 406/500

Epoch 00406: val_acc did not improve from 0.97000
38/38 - 0s - loss: 1.6566e-04 - acc: 0.9383 - val_loss: 1.4028e-04 - val_acc: 0.9325 - 59ms/epoch - 2ms/step
Epoch 407/500

Epoch 00407: val_acc did not improve from 0.97000
38/38 - 0s - loss: 1.0541e-04 - acc: 0.9533 - val_loss: 1.1806e-04 - val_acc: 0.9475 - 63ms/epoch - 2ms/step
Epoch 408/500

Epoch 00408: val_acc did not improve from 0.97000
38/38 - 0s - loss: 7.7816e-05 - acc: 0.9625 - val_loss: 8.2134e-05 - val_acc: 0.9550 - 62ms/epoch - 2ms/step
Epoch 409/500

Epoch 00409: val_acc did not improve from 0.97000
38/38 - 0s - loss: 7.2057e-05 - acc: 0.9592 - val_loss: 8.4234e-0

Epoch 451/500

Epoch 00451: val_acc did not improve from 0.97500
38/38 - 0s - loss: 3.9055e-05 - acc: 0.9683 - val_loss: 4.2583e-05 - val_acc: 0.9650 - 82ms/epoch - 2ms/step
Epoch 452/500

Epoch 00452: val_acc did not improve from 0.97500
38/38 - 0s - loss: 3.2460e-05 - acc: 0.9683 - val_loss: 3.9190e-05 - val_acc: 0.9625 - 107ms/epoch - 3ms/step
Epoch 453/500

Epoch 00453: val_acc did not improve from 0.97500
38/38 - 0s - loss: 3.3087e-05 - acc: 0.9683 - val_loss: 3.7154e-05 - val_acc: 0.9650 - 110ms/epoch - 3ms/step
Epoch 454/500

Epoch 00454: val_acc did not improve from 0.97500
38/38 - 0s - loss: 3.2261e-05 - acc: 0.9658 - val_loss: 3.5250e-05 - val_acc: 0.9575 - 111ms/epoch - 3ms/step
Epoch 455/500

Epoch 00455: val_acc did not improve from 0.97500
38/38 - 0s - loss: 5.3152e-05 - acc: 0.9650 - val_loss: 1.1487e-04 - val_acc: 0.9600 - 119ms/epoch - 3ms/step
Epoch 456/500

Epoch 00456: val_acc did not improve from 0.97500
38/38 - 0s - loss: 1.1911e-04 - acc: 0.9600 - val_loss: 1.322

Epoch 498/500

Epoch 00498: val_acc did not improve from 0.97750
38/38 - 0s - loss: 3.0254e-05 - acc: 0.9708 - val_loss: 3.1949e-05 - val_acc: 0.9625 - 56ms/epoch - 1ms/step
Epoch 499/500

Epoch 00499: val_acc did not improve from 0.97750
38/38 - 0s - loss: 5.2594e-05 - acc: 0.9633 - val_loss: 6.0728e-05 - val_acc: 0.9650 - 56ms/epoch - 1ms/step
Epoch 500/500

Epoch 00500: val_acc did not improve from 0.97750
38/38 - 0s - loss: 5.8570e-05 - acc: 0.9658 - val_loss: 7.4872e-05 - val_acc: 0.9525 - 57ms/epoch - 2ms/step
